In [1]:
import requests
from urllib import request
from bs4 import BeautifulSoup
import time
import pandas as pd
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
from requests import Session
import re
import datetime
from tqdm import tqdm
# from kafka import KafkaProducer
# import kafka
import random
import schedule

## 종목별 코드

In [2]:
ACODE = {"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
            "에코프로":"086520","신성이엔지":"011930","유니슨":"018000",
            "카카오":"035720","네이버":"035420","엔씨소프트":"036570",
            "sds":"018260","삼성전자":"005930","SK하이닉스":"000660",
            "skc":"011790","한솔케미칼":"014680","DB하이텍":"000990",
            "롯데쇼핑":"023530","신세계":"004170","삼성물산":"028260",
            "동서":"026960","kb금융":"105560","한국금융지주":"071050","신한지주":"055550",
            "우리금융지주":"316140","하나금융지주":"086790",
         "넥슨지티":"041140","한빛소프트":"047080","코세스":"089890",
      "드래곤플라이":"030350","에이트원":"230980","엔텔스":"069410"}

## 함수 정의

In [3]:
TIME_MAPPING = {}

i = 1
START = datetime.datetime(1,1,1,8,55) # test


s = START
while s.strftime('%H:%M') < "16:00":
    e = s + datetime.timedelta(minutes = 4)
    TIME_MAPPING[i] = (s.time(), e.time())

    s += datetime.timedelta(minutes=5)
    i += 1

TIME_MAPPING[86] = (s.time(), )

#TIME_STOP = TIME_MAPPING[current]

In [ ]:
# TIME_MAPPING

In [4]:
def title_crawl(p_l, ticker, ts):

    # global TIME_STOP 
    global HEADERS
    global result
    global OPINION

    for i in range(0,len(p_l)-1):
        response = requests.get('https://gall.dcinside.com' + str(p_l[i]), headers = HEADERS)
        soup = BeautifulSoup(response.content,'html.parser')
        items = soup.find_all("table",{'class':'gall_list'})
        contents = soup.find('tbody').find_all('tr')
        
        for j in contents:         
            if j.find('td',{'class':'gall_writer ub-writer'}).text=='운영자':
                pass
            else:
                # 날짜
                date_dict = j.find('td',{'class':'gall_date'}).attrs
                    
                if date_dict['title'].split()[1][:-3] < str(ts[0]) or date_dict['title'].split()[1][:-3] > str(ts[1]):
                    print('out of range')
                    # return -1
                    break
                else:
                    # 제목
                    title = j.find('a').text
                    # 조회수
                    views_tag = j.find('td', class_='gall_count')
                    view = views_tag.text
                    view=int(view)
                    put_data = (title,view)
                    

                    result.append(put_data)
                    # print(result)
            # ya[ACODE]= result
        # return ya
        

In [10]:
def DC_crawling(ticker, page, ct, ts):
    global OPINION
    global HEADERS

    result = []
    url = f"https://gall.dcinside.com/board/lists?id=neostock&s_type=search_subject_memo&s_keyword={ticker}"

    a = f'/board/lists?id=neostock&s_type=search_subject_memo&s_keyword={ticker}'

    response = requests.get(url,headers=HEADERS)
    soup = BeautifulSoup(response.text,'html.parser')
    items = soup.find('div',{'class':"bottom_paging_box"})

    # 페이지
    page_list =[a]
    for i in items.find_all('a'):
        p = i['href']
        page_list.append(p) 

    #  첫페이지
    title_crawl(page_list, ticker, ts)

    k = 0
    while k < page :
        response = requests.get('https://gall.dcinside.com' + str(page_list[-1]),headers=HEADERS)
        soup = BeautifulSoup(response.content,'html.parser')
        items = soup.find('div',{'class':"bottom_paging_box"})
        page_list=[str(page_list[-1])]

        for i in items.find_all('a'):
            pp = i['href']
            page_list.append(pp)
        page_list.pop(1) 
        
        if title_crawl(page_list, ticker, ts) == -1:
            break
        else:
            k += 1

    # 페이지 다 돌았으면 OPINION에 넣기
    OPINION[ct] = result

    # return result

In [ ]:
# for t in ACODE.keys():
#     DC_crawling(t, 10 ,TIME_MAPPING)

In [11]:

bootstrap_servers = ["localhost:9092"]#, "localhost:9093"]  # kafka broker ip
topicName = 'DC_test'
# producer = kafka.KafkaProducer(bootstrap_servers=bootstrap_servers)

HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
OPINION = {i : [] for i in range(1, 87)}
def msg():
    #스케줄링
    result = []
    current = 1
    global TIME_STOP
    TIME_STOP = TIME_MAPPING[current]
    
    while True:
        start = time.time()
        for t in tqdm(ACODE.keys()): DC_crawling(t, 10, current, TIME_STOP)
        end = time.time()
        print(end-start)
        if TIME_MAPPING[current][0].strftime('%H:%M') <= datetime.datetime.now().strftime('%H:%M'):
            # producer.send(topicName, str(result).encode())
            print(result)
            current+=1
            result = []
        if current == 85:
            break
        #tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        #producer.send(topicName, tim.encode())
# schedule.every().day.at("21:44").do(msg)
# while True:
    
    # schedule.run_pending()
    

In [12]:
msg()

 10%|█         | 3/30 [00:16<02:30,  5.57s/it]

out of range
out of range
out of range
out of range
out of range


 13%|█▎        | 4/30 [00:22<02:26,  5.62s/it]

out of range


 17%|█▋        | 5/30 [00:27<02:14,  5.38s/it]

...

out of range


100%|██████████| 30/30 [03:04<00:00,  6.15s/it]


184.55609583854675
[]


 10%|█         | 3/30 [00:17<02:38,  5.88s/it]


KeyboardInterrupt: 

In [14]:
result

NameError: name 'result' is not defined

In [ ]:
bootstrap_servers = ['localhost:9092']  # kafka broker ip
topicName = 'DC'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)


for value in ACODE.keys():
    print(value)
    DC_con = DC_crawling(value, 100)
    

    for i in DC_con:
        producer.send(topicName, str(i).encode())      
        tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        metadata = producer.send(topicName, tim.encode())
        print(i," - ",metadata)
#producer.flush()

#        time.sleep(1)



In [ ]:
네이버, DC 각 1개씩 프로듀서
토픽은 동일.
파티션은 두개
파티션의 키 지정해서 넣어본 적 있으세요?

